# Тестовое задание ПАО "Газпром"

## Описание ситуации
**Имеется набор файлов от 11 трансгазов[ Газотранспортные операторы, собственники газопроводов высокого давления, заканчивающихся выходом на ГРС. Для примера приложено только 2 отчета (ГТ Югорск и ГТ Чайковский), остальные сведены в flat_table.xls.] с указанием фактической товаротранспортной работы за январь 2018 г. в разрезе:**

(1) газа, поставляемого данным трансгазом[ Только для ГТ Югорск и ГТ Сургут. Также есть ГРС (потребители), получающие газ напрямую от этих двух трансгазов, т.е. без транзита по другим трансгазам.] от балансового пункта (Надым и Локосово[ Надым=«КС Надым (216,2 км газопровода Уренгой-Петровск)», а Локосово= «БП "622,5 км" (Локосово) (622,5 км газопровода Уренгой-Челябинск)».]) до границы со следующим трансгазом;\
(2) транзитного газа для передачи следующему по цепочке трансгазу;\
(3) газа, поставленного данным трансгазом на газораспределительную станцию (ГРС[ Все значения столбца tg_to_grs по газу типа «Газ, для последующей реализации, переданный на ГРС Исполнителя и /или на границе газопроводов Исполнителя с газопроводами-отводами сторонней ГТО» считаются названиями ГРС.]) для последующей поставки конечному потребителю по сетям среднего давления.

Для каждого вида газов указано расстояние транспортировки по газопроводу данного трансгаза.

*Весь набор файлов для удобства уже преобразован в плоскую таблицу, но не обработан, т.е. названия не приведены к единообразию.*

## Задача

**В любой удобной программной среде (R, python, etc.) написать скрипт, который для каждой ГРС указывал бы наиболее короткий (в случае вариативности) фактический маршрут транспортировки от балансового пункта по форме таблицы в Приложении. В перспективе данный скрипт должен обрабатывать аналогичные по формату месячные данные за произвольный период.**

*Справочно: Балансовые пункты считаются источником газа, они расположены в двух трансгазах: Надым в ГТ Югорск и Локосово в ГТ Сургут.  Транспортировка газа до балансового пункта находится вне области анализа данной задачи.*

1 Газотранспортные операторы, собственники газопроводов высокого давления, заканчивающихся выходом на ГРС. Для примера приложено только 2 отчета (ГТ Югорск и ГТ Чайковский), остальные сведены в flat_table.xls.\
2 Только для ГТ Югорск и ГТ Сургут. Также есть ГРС (потребители), получающие газ напрямую от этих двух трансгазов, т.е. без транзита по другим трансгазам.
3 Надым=«КС Надым (216,2 км газопровода Уренгой-Петровск)», а Локосово= «БП "622,5 км" (Локосово) (622,5 км газопровода Уренгой-Челябинск)».\
4 Все значения столбца tg_to_grs по газу типа «Газ, для последующей реализации, переданный на ГРС Исполнителя и /или на границе газопроводов Исполнителя с газопроводами-отводами сторонней ГТО» считаются названиями ГРС.

## 1. Подгрузка необходимых библиотек

In [123]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

## 2. Загрузка и очистка данных

Подгрузим плоскую таблицу (датасет) и осмотрим первые 10 строк на предмет корректности загрузки.\
*Примечание: Исходная таблица "flat_table.xlsx" должна находиться в одном каталоге с исполняемым .ipynb файлом, иначе необходимо предусмотреть иной способ подгрузки файла (с Google-диска, из сторонней директории и т.д.)*

In [124]:
df = pd.read_excel('flat_table.xlsx')
df.sort_values(by=['tg_to_grs'], inplace=True)
df.head(10)

,gas_type,date,tg_owner,tg_from,tg_to_grs,distance
132,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Чайковский""","ООО ""ГТ Югорск""",44 км г/о Очер-Кудымкар купрос*,342.100
32,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Казань""","ООО ""ГТ Чайковский""","АГРС АО ""Аммоний""*",16.200
149,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Югорск""","КС Надым (216,2 км газопровода Уренгой-Петровск)",АГРС НТГРЭС*,1181.800
19,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Екатеринбург""","ООО ""ГТ Югорск""",Алапаевск*,121.900
65,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Нижний Новгород""","ООО ""ГТ Казань""",Алексеевская,463.570
0,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Екатеринбург""","ООО ""ГТ Сургут""","Асбест, ГРС-2",506.900
29,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Казань""","ООО ""ГТ Самара""",Ашальчи*,138.735
121,Транспортировка до Балансового пункта,2018-01-01,"ООО ""Газпром трансгаз Сургут""",Газопровод внешнего транспорта Комсомольского ...,"БП ""622,5 км"" (Локосово) (622,5 км газопровода...",414.200
119,Транспортировка до Балансового пункта,2018-01-01,"ООО ""Газпром трансгаз Сургут""",Газопровод внешнего транспорта Вынгаяхинского ...,"БП ""622,5 км"" (Локосово) (622,5 км газопровода...",351.250
122,Транспортировка до Балансового пункта,2018-01-01,"ООО ""Газпром трансгаз Сургут""","622 км Уренгой-Челябинск (ООО ""Газ-Ойл трейдинг"")","БП ""622,5 км"" (Локосово) (622,5 км газопровода...",NaN


In [125]:
print(f'Размерность датасета: {df.shape}')
print('==============================')
print(df.info())

Размерность датасета: (192, 6)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 192 entries, 132 to 134
Data columns (total 6 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   gas_type   192 non-null    object        
 1   date       192 non-null    datetime64[ns]
 2   tg_owner   192 non-null    object        
 3   tg_from    192 non-null    object        
 4   tg_to_grs  192 non-null    object        
 5   distance   191 non-null    float64       
dtypes: datetime64[ns](1), float64(1), object(4)
memory usage: 10.5+ KB
None


В задании указано, что газ, транспортированный до балансового пункта, не участвует в анализе задачи. Удалим все строки, касающиеся данного вида газа.

In [126]:
df['gas_type'].value_counts()

Газ, для последующей реализации, переданный на ГРС Исполнителя и /или на границе газопроводов Исполнителя с газопроводами-отводами сторонней ГТО    158
Транзитный газ                                                                                                                                       28
Транспортировка до Балансового пункта                                                                                                                 6
Name: gas_type, dtype: int64

In [127]:
df.drop(df[df['gas_type'] ==
        'Транспортировка до Балансового пункта'].index, inplace=True)
print(df['gas_type'].value_counts())
print('============================')
print(f'Размерность: {df.shape}')

Газ, для последующей реализации, переданный на ГРС Исполнителя и /или на границе газопроводов Исполнителя с газопроводами-отводами сторонней ГТО    158
Транзитный газ                                                                                                                                       28
Name: gas_type, dtype: int64
Размерность: (186, 6)


Приведем все наименования трансгазов к единообразию. За основу возьмем значения из столбца "tg_owner".

In [128]:
df['tg_owner'].value_counts()

ООО "Газпром трансгаз Нижний Новгород"    32
ООО "Газпром трансгаз Москва"             29
ООО "Газпром трансгаз Екатеринбург"       25
ООО "Газпром трансгаз Чайковский"         16
ООО "Газпром трансгаз Санкт-Петербург"    16
ООО "Газпром трансгаз Югорск"             15
ООО "Газпром трансгаз Самара"             15
ООО "Газпром трансгаз Казань"             11
ООО "Газпром трансгаз Уфа"                 8
ООО "Газпром трансгаз Томск"               5
ООО "Газпром трансгаз Ухта"                4
ООО "Газпром трансгаз Саратов"             4
ООО "Газпром трансгаз Сургут"              3
ООО "Газпром трансгаз Краснодар"           1
ООО "Газпром трансгаз Ставрополь"          1
ООО "Газпром трансгаз Волгоград"           1
Name: tg_owner, dtype: int64

Посмотрим, как выглядят трансгазы в "tg_from" и в "tg_to_grs".

In [129]:
df['tg_from'].value_counts()

ООО "ГТ Н. Новгород"                                                 30
ООО "ГТ Чайковский"                                                  27
ООО "ГТ Югорск"                                                      26
ООО "ГТ Сургут"                                                      24
ООО "ГТ Ухта"                                                        16
КС Надым (216,2 км газопровода Уренгой-Петровск)                     15
ООО «ГТ Казань»                                                      15
ООО "ГТ Казань"                                                      11
ООО "ГТ Екатеринбург"                                                 7
ООО "ГТ Самара"                                                       4
БП "622,5 км" (Локосово) (622,5 км газопровода Уренгой-Челябинск)     3
ООО "ГТ Москва"                                                       3
ООО "ГТ Ставрополь"                                                   1
ООО "ГТ С-Петербург"                                            

In [130]:
df['tg_to_grs'].value_counts()

ООО "ГТ Н. Новгород"                 3
ООО «ГТ Казань»                      3
ООО "ГТ Москва"                      2
ООО "ГТ С-Петербург"                 2
ООО "ГТ Саратов"                     2
                                    ..
КС Нижняя Тура (на ГТ Чайковский)    1
КС Приполярная (на ГТ Ухта)          1
КСПГ ЗАО "Криогаз"*                  1
Кабаково                             1
г/о ПАО "Метафракс"*                 1
Name: tg_to_grs, Length: 177, dtype: int64

Видим некорректный ввод данных: сокращения, разные кавычки и проч. Приведем все к единообразию. Возможно, это обусловлено ручным вводом данных, а так же поступлением данных из разных источников.

Заменим кавычки на английский тип.

In [131]:
df.replace(to_replace='«', value='"', inplace=True, regex=True)
df.replace(to_replace='»', value='"', inplace=True, regex=True)
df['tg_to_grs'].value_counts()

ООО "ГТ Казань"                        5
ООО "ГТ Н. Новгород"                   3
ООО "ГТ Саратов"                       3
ООО "ГТ Москва"                        2
ООО "ГТ С-Петербург"                   2
                                      ..
КС Нижняя Тура (на ГТ Екатеринбург)    1
КС Нижняя Тура (на ГТ Чайковский)      1
КС Приполярная (на ГТ Ухта)            1
КСПГ ЗАО "Криогаз"*                    1
г/о ПАО "Метафракс"*                   1
Name: tg_to_grs, Length: 175, dtype: int64

Выделим в отдельный столбец все города трансгазов, а затем соберем их в множество, удалив дубликаты.

In [132]:
pattern = r'ООО "Газпром трансгаз ([А-Яа-я\s-]+)"'
df['city'] = df['tg_owner'].str.extract(pattern, expand=False)
cities = set(df['city'].tolist())
print(cities)
print('=====================================')
print(f'Размер перечня городов: {len(cities)} элементов.')

{'Санкт-Петербург', 'Ухта', 'Саратов', 'Нижний Новгород', 'Екатеринбург', 'Югорск', 'Москва', 'Казань', 'Томск', 'Волгоград', 'Краснодар', 'Самара', 'Сургут', 'Ставрополь', 'Чайковский', 'Уфа'}
Размер перечня городов: 16 элементов.


Найдем все вхождения названий городов в столбце "tg_from" и переименуем их по образцу, затем проверим результат:

In [133]:
for city in cities:
    df.loc[df['tg_from'].str.contains(
        '"ГТ '+city), 'tg_from'] = 'ООО "Газпром трансгаз '+city+'"'
df['tg_from'].value_counts()

ООО "ГТ Н. Новгород"                                                 30
ООО "Газпром трансгаз Чайковский"                                    27
ООО "Газпром трансгаз Югорск"                                        26
ООО "Газпром трансгаз Казань"                                        26
ООО "Газпром трансгаз Сургут"                                        24
ООО "Газпром трансгаз Ухта"                                          16
КС Надым (216,2 км газопровода Уренгой-Петровск)                     15
ООО "Газпром трансгаз Екатеринбург"                                   7
ООО "Газпром трансгаз Самара"                                         4
БП "622,5 км" (Локосово) (622,5 км газопровода Уренгой-Челябинск)     3
ООО "Газпром трансгаз Москва"                                         3
ООО "Газпром трансгаз Ставрополь"                                     1
ООО "ГТ С-Петербург"                                                  1
ООО "Газпром трансгаз Уфа"                                      

Есть элементы, которые по скрипту не переименовались ("Нижний Новгород" и "Санкт-Петербург"), но т.к. их всего два, переименуем их вручную.

In [134]:
df.loc[df['tg_from'] == 'ООО "ГТ С-Петербург"',
       'tg_from'] = 'ООО "Газпром трансгаз Санкт-Петербург"'
df.loc[df['tg_from'] == 'ООО "ГТ Н. Новгород"',
       'tg_from'] = 'ООО "Газпром трансгаз Нижний Новгород"'
df['tg_from'].value_counts()

ООО "Газпром трансгаз Нижний Новгород"                               30
ООО "Газпром трансгаз Чайковский"                                    27
ООО "Газпром трансгаз Югорск"                                        26
ООО "Газпром трансгаз Казань"                                        26
ООО "Газпром трансгаз Сургут"                                        24
ООО "Газпром трансгаз Ухта"                                          16
КС Надым (216,2 км газопровода Уренгой-Петровск)                     15
ООО "Газпром трансгаз Екатеринбург"                                   7
ООО "Газпром трансгаз Самара"                                         4
БП "622,5 км" (Локосово) (622,5 км газопровода Уренгой-Челябинск)     3
ООО "Газпром трансгаз Москва"                                         3
ООО "Газпром трансгаз Ставрополь"                                     1
ООО "Газпром трансгаз Санкт-Петербург"                                1
ООО "Газпром трансгаз Уфа"                                      

Теперь четко видно трансгазы и балансовые пункты ("Надым" и "Локосово").\
Очистка данных в столбце "tg_from" произведена.\
Обработаем данные в столбце "tg_to_grs".

In [135]:
df['tg_to_grs'].value_counts()

ООО "ГТ Казань"                        5
ООО "ГТ Н. Новгород"                   3
ООО "ГТ Саратов"                       3
ООО "ГТ Москва"                        2
ООО "ГТ С-Петербург"                   2
                                      ..
КС Нижняя Тура (на ГТ Екатеринбург)    1
КС Нижняя Тура (на ГТ Чайковский)      1
КС Приполярная (на ГТ Ухта)            1
КСПГ ЗАО "Криогаз"*                    1
г/о ПАО "Метафракс"*                   1
Name: tg_to_grs, Length: 175, dtype: int64

In [136]:
for city in cities:
    df.loc[df['tg_to_grs'].str.contains(
        '"ГТ '+city), 'tg_to_grs'] = 'ООО "Газпром трансгаз '+city+'"'
df.loc[df['tg_to_grs'] == 'ООО "ГТ С-Петербург"',
       'tg_to_grs'] = 'ООО "Газпром трансгаз Санкт-Петербург"'
df.loc[df['tg_to_grs'] == 'ООО "ГТ Н. Новгород"',
       'tg_to_grs'] = 'ООО "Газпром трансгаз Нижний Новгород"'
df['tg_to_grs'].value_counts()

ООО "Газпром трансгаз Казань"             5
ООО "Газпром трансгаз Нижний Новгород"    3
ООО "Газпром трансгаз Саратов"            3
ООО "Газпром трансгаз Москва"             2
ООО "Газпром трансгаз Санкт-Петербург"    2
                                         ..
КС Нижняя Тура (на ГТ Екатеринбург)       1
КС Нижняя Тура (на ГТ Чайковский)         1
КС Приполярная (на ГТ Ухта)               1
КСПГ ЗАО "Криогаз"*                       1
г/о ПАО "Метафракс"*                      1
Name: tg_to_grs, Length: 175, dtype: int64

В столбце "tg_to_grs" имеются значения конструкции по тику 'КС ХХХ (на ГТ XXXX'), что препятствует нормальной фильтрации значений.\
В рамках данной задачи приведем данные значения к принятой выше конструкции опираясь на направление трансгаза.

In [137]:
df.loc[df['tg_to_grs'] ==
       'КС Карпинская (на ГТ Чайковский)', 'tg_to_grs'] = 'ООО "Газпром трансгаз Чайковский"'
df.loc[df['tg_to_grs'] ==
       'КС Лялинская (на ГТ Чайковский)', 'tg_to_grs'] = 'ООО "Газпром трансгаз Чайковский"'
df.loc[df['tg_to_grs'] ==
       'КС Нижняя Тура (на ГТ Чайковский)', 'tg_to_grs'] = 'ООО "Газпром трансгаз Чайковский"'
df.loc[df['tg_to_grs'] == 'КС Нижняя Тура (на ГТ Екатеринбург)',
       'tg_to_grs'] = 'ООО "Газпром трансгаз Екатеринбург"'
df.loc[df['tg_to_grs'] ==
       'КС Приполярная (на ГТ Ухта)', 'tg_to_grs'] = 'ООО "Газпром трансгаз Ухта"'

Проверим, что названия ГРС не "утекли" в полные названия трансгазов. За тестовый образец возьмем ГРС "Казань №5".

In [138]:
df.loc[df['tg_to_grs'] == 'Казань №5']

,gas_type,date,tg_owner,tg_from,tg_to_grs,distance,city
27,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Казань""","ООО ""Газпром трансгаз Нижний Новгород""",Казань №5,9.76,Казань


Удалим временный столбец df['city'].

In [139]:
df.drop(columns='city', inplace=True)
df.head(5)

,gas_type,date,tg_owner,tg_from,tg_to_grs,distance
132,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Чайковский""","ООО ""Газпром трансгаз Югорск""",44 км г/о Очер-Кудымкар купрос*,342.10
32,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Казань""","ООО ""Газпром трансгаз Чайковский""","АГРС АО ""Аммоний""*",16.20
149,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Югорск""","КС Надым (216,2 км газопровода Уренгой-Петровск)",АГРС НТГРЭС*,1181.80
19,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Екатеринбург""","ООО ""Газпром трансгаз Югорск""",Алапаевск*,121.90
65,"Газ, для последующей реализации, переданный на...",2018-01-01,"ООО ""Газпром трансгаз Нижний Новгород""","ООО ""Газпром трансгаз Казань""",Алексеевская,463.57


*Опционально: Выгрузим очищенный датафрейм в .xlsx файл.*

In [140]:
# df.to_excel('clean_data.xlsx', sheet_name='query') # Локальная выгрузка в эксель для проверки (для запуска ячейки - удалить #).

## 3. Построение скрипта определения маршрута.

Создадим список исключений (список промежуточных ГРС). Строки со значениями в столбце "tg_to_grs" из этого списка будут игнорироваться и не помещаться в новый датафрейм.

In [141]:
list_of_exceptions = list(df['tg_from'].unique())
list_of_exceptions.remove(
    'БП "622,5 км" (Локосово) (622,5 км газопровода Уренгой-Челябинск)')
list_of_exceptions.remove('КС Надым (216,2 км газопровода Уренгой-Петровск)')
add_list = ['ООО "Газпром трансгаз Краснодар"', 'ООО "Газпром трансгаз Томск"'] # В перечне ГРС в столбце 'tg_to_grs' имеются промежуточные ГРС, которых нет в 'tg_from'
list_of_exceptions.extend(add_list)
list_of_exceptions.sort()
list_of_exceptions

['ООО "Газпром трансгаз Волгоград"',
 'ООО "Газпром трансгаз Екатеринбург"',
 'ООО "Газпром трансгаз Казань"',
 'ООО "Газпром трансгаз Краснодар"',
 'ООО "Газпром трансгаз Москва"',
 'ООО "Газпром трансгаз Нижний Новгород"',
 'ООО "Газпром трансгаз Самара"',
 'ООО "Газпром трансгаз Санкт-Петербург"',
 'ООО "Газпром трансгаз Саратов"',
 'ООО "Газпром трансгаз Ставрополь"',
 'ООО "Газпром трансгаз Сургут"',
 'ООО "Газпром трансгаз Томск"',
 'ООО "Газпром трансгаз Уфа"',
 'ООО "Газпром трансгаз Ухта"',
 'ООО "Газпром трансгаз Чайковский"',
 'ООО "Газпром трансгаз Югорск"']

Создадим список остановки перебора (конечные (начальные по смыслу) пункты). Если значение в столбце исходного датафрейма будет в этом списке, цикл перебора остановится (это будет значить, что маршрут достиг БП).

In [142]:
stop_list = ['КС Надым (216,2 км газопровода Уренгой-Петровск)',
             'БП "622,5 км" (Локосово) (622,5 км газопровода Уренгой-Челябинск)']

In [143]:
new_df_data = []  # Список для хранения данных нового датафрейма

# Проходим циклом по всем строкам исходной таблицы df, чтобы отобрать строки для нового датафрейма new_df
for index, row in df.iterrows():
    tg_to_grs = row['tg_to_grs']
    if tg_to_grs in list_of_exceptions: # Проверка на принадлежность 'tg_to_grs' к промежуточным ГРС (если прнадлежность подтверждается - пропускаем такую строку)
        continue
    tg_owner = row['tg_owner']
    tg_dist = row['distance']
    
    new_row = {'ГРС': tg_to_grs, 'ГТ-0 (собственник)': tg_owner, 'расстояние_1': tg_dist}
    new_df_data.append(new_row)

new_df = pd.DataFrame(new_df_data)

new_df.sort_values(by=['ГРС'], inplace=True) # Сортируем датафрейм по ГРС (не обязательно)

# Проходим циклов по каждой строке полученного датафрейма для добавления новых столбцов
for index, row in new_df.iterrows():
    column_index = 1 
    tg_to_grs = row['ГРС']
    tg_owner = row['ГТ-0 (собственник)']
    tg_dist = df.loc[df['tg_to_grs'] == tg_to_grs, 'distance'].values[0]
    tg_from = df.loc[df['tg_to_grs'] == tg_to_grs, 'tg_from'].values[0]
    flag = True
    
    while flag:
        column_grs = f'ГТ-{column_index}'
        column_dist = f'расстояние_{column_index}'
        
        if tg_from in stop_list: # Проверка на наличие tg_from в списке конечных пунктов
            new_df.at[index, column_dist] = tg_dist
            new_df.at[index, column_grs] = tg_from
            flag = False
        else:
            new_df.at[index, column_dist] = tg_dist 
            new_df.at[index, column_grs] = tg_from 
            
            # Определяем новую активную строку исходя из условия, что tg_owner == tg_from, a tg_to_grs == tg_owner
            next_row = df.loc[(df['tg_owner'] == tg_from) & (df['tg_to_grs'] == tg_owner)]
            if len(next_row) > 1:
                next_row = next_row.loc[next_row['distance'] == next_row['distance'].min()] # Из нескольких одинаковых строк выбираем строку с наименьшим distance
            
            if len(next_row) == 0: # Если такой строки не находится, заканчиваем цикл
                flag = False
                break
            # Производим переназначение переменных исходя из новой активной строки
            tg_from = next_row['tg_from'].values[0]
            tg_to_grs = next_row['tg_to_grs'].values[0]
            tg_owner = next_row['tg_owner'].values[0]
            tg_dist = next_row['distance'].values[0]
            column_index += 1 # Увеличиваем индекс столбца на 1

new_df['Итого'] = new_df.filter(regex='^расстояние').sum(axis=1).round(2) # Подсчет всех расстояний (с округлением до двух знаков после запятой)

**Локальная проверка значений.**\
Проверим значения ГРС из примера в описании задания.\
Это были ГРС: "Тамбов, ГРС-2А" и "Реж".

In [144]:
new_df.loc[new_df['ГРС'] == "Реж"]

,ГРС,ГТ-0 (собственник),расстояние_1,ГТ-1,расстояние_2,ГТ-2,расстояние_3,ГТ-3,расстояние_4,ГТ-4,...,ГТ-5,расстояние_6,ГТ-6,расстояние_7,ГТ-7,расстояние_8,ГТ-8,расстояние_9,ГТ-9,Итого
117,Реж,"ООО ""Газпром трансгаз Екатеринбург""",552.0,"ООО ""Газпром трансгаз Сургут""",863.5,"БП ""622,5 км"" (Локосово) (622,5 км газопровода...",NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1415.5


In [145]:
new_df.loc[new_df['ГРС'] == 'Тамбов, ГРС-2А']

,ГРС,ГТ-0 (собственник),расстояние_1,ГТ-1,расстояние_2,ГТ-2,расстояние_3,ГТ-3,расстояние_4,ГТ-4,...,ГТ-5,расстояние_6,ГТ-6,расстояние_7,ГТ-7,расстояние_8,ГТ-8,расстояние_9,ГТ-9,Итого
140,"Тамбов, ГРС-2А","ООО ""Газпром трансгаз Саратов""",12.9,"ООО ""Газпром трансгаз Москва""",71.3,"ООО ""Газпром трансгаз Нижний Новгород""",532.35,"ООО ""Газпром трансгаз Казань""",149.65,"ООО ""Газпром трансгаз Чайковский""",...,"ООО ""Газпром трансгаз Югорск""",1136.9,"КС Надым (216,2 км газопровода Уренгой-Петровск)",NaN,NaN,NaN,NaN,NaN,NaN,2556.72


*Опционально: Выгрузим готовый датафрейм с построенными маршрутами в .xlsx файл. По-умолчанию файл сохраняется в директорию с исполняемым .ipynb файлом.*

In [146]:
# new_df.to_excel('transgases.xlsx') # Для выгрузки .xlsx файла необходимо убрать # перед кодом и запустить ячейку.

## 4. Визуализация

Для визуализации необходимо доустановить библиотеку ipywidgets (можно упаковать весь скрипт вместе с библиотеками в виртуальное окружение и нужные библиотеки будут подгружаться одной командой локально).

In [147]:
pip install ipywidgets

Note: you may need to restart the kernel to use updated packages.


Построение визуализации датафрейма (отображение одной или нескольких интересующих пользователя строк исходя из выбранных ГРС).\
*Примечание: Выбор нескольких ГРС производится через клавишу command (MAC) или shift(Windows).*

In [148]:
import plotly.graph_objects as go
import ipywidgets as widgets
from IPython.display import display

# Функция для обработки события выбора строки из выпадающего списка
def select_row(change):
    selected_rows = new_df[new_df['ГРС'].isin(drop_menu.value)]
    selected_rows = selected_rows.dropna(axis=1, how='all') # Удаление столбцов с пустыми значениями
    frame.data[0].header.values = selected_rows.columns # Обновление заголовков столбцов
    frame.data[0].cells.values = [selected_rows[col] for col in selected_rows.columns]

# Создание выпадающего меню на основе столбца "ГРС"
drop_menu = widgets.SelectMultiple(
    options=new_df['ГРС'].tolist(),
    description='Выбор ГРС:'
)

# Привязка функции обновления к событию изменения значения выпадающего меню
drop_menu.observe(select_row, names='value')

# Создание первой пустой строки (до момета выбора ГРС)
empty_df = pd.DataFrame(columns=new_df.columns)

frame = go.FigureWidget([
    go.Table(
        header=dict(values=empty_df.columns),
        cells=dict(values=[empty_df[col] for col in empty_df.columns])
    )
])
# Отображение выпадающего меню и строки(-ок)
display(drop_menu)
display(frame)

SelectMultiple(description='Выбор ГРС:', options=('44 км г/о Очер-Кудымкар купрос*', 'АГРС АО "Аммоний"*', 'АГ…

FigureWidget({
    'data': [{'cells': {'values': [[], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], [], []]},
              'header': {'values': array(['ГРС', 'ГТ-0 (собственник)', 'расстояние_1', 'ГТ-1', 'расстояние_2',
                                          'ГТ-2', 'расстояние_3', 'ГТ-3', 'расстояние_4', 'ГТ-4', 'расстояние_5',
                                          'ГТ-5', 'расстояние_6', 'ГТ-6', 'расстояние_7', 'ГТ-7', 'расстояние_8',
                                          'ГТ-8', 'расстояние_9', 'ГТ-9', 'Итого'], dtype=object)},
              'type': 'table',
              'uid': '2cd45a1d-f8c1-463e-88df-b29f224f59ed'}],
    'layout': {'template': '...'}
})